In [68]:
# imports
import xml.etree.ElementTree as Xet # for parsing and creating XML data
import pandas as pd
import os, csv, re, nltk
from flair.data import Corpus # in order to use the functions tha flair has
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings # these embeddings helps NER to perform better
from itertools import islice
from nltk.stem import WordNetLemmatizer # previously need to download "nltk.download('wordnet')" and "nltk.download('omw-1.4')". But beware if new version comes out
from tqdm import tqdm # to display loop in a bar
from openie import StanfordOpenIE # for using our OIE tool
from nltk.tokenize import sent_tokenize
from googletrans import Translator # don't forget to run "!pip install googletrans==3.1.0a0" before using this

### NER (Named-Entity Recognition)

In [183]:
# functions
# -*- coding: utf-8 -*-

# English: en, German: de, French: fr, ... -> creates the tsv of given descriptor of any language
def create_tsv_of_language(given_language):
    """
    Before running this function below, the "desc_"".xml" file (that is downloaded from EuroVoc website) needs to be downloaded and added to package "data/""/descriptors/..."
    """
    cols = ['ID', given_language.upper()] # will be saving in a tsv with ids and their corresponding terms
    rows = []

    # parsing the xml file -> with the given EuroVoc descriptors
    temp_path = os.getcwd()
    temp_path = temp_path.replace("src\\main", "data\\" + given_language + "\\descriptors\\desc_" + given_language + ".xml")
    xml_parse = Xet.parse(temp_path)
    root = xml_parse.getroot()

    # iterate through the elements of xml file
    for element in root:
        rows.append({"ID": element.find("DESCRIPTEUR_ID").text, given_language.upper(): element.find("LIBELLE").text})

    # creating the tsv file
    df = pd.DataFrame(rows, columns=cols)
    df.to_csv("eurovoc_" + given_language + ".tsv", sep='\t', index=False) # using sep='\t' gives us a tsv file instead of csv

def create_tsv_of_any_given_concept(given_concept_dict, given_language): # requires a dict in style of {EN:..., ID:...}
    cols = ['ID', given_language.upper()] # will be saving in a tsv with ids and their corresponding terms
    rows = []

    # iterate through the elements of xml file
    for key, value in given_concept_dict.items():
        rows.append({"ID": value, given_language.upper(): key})

    # creating the tsv file
    df = pd.DataFrame(rows, columns=cols)
    df.to_csv("updated_eurovoc_" + given_language + ".tsv", sep='\t', index=False) # using sep='\t' gives us a tsv file instead of csv

# this function assumes we get the text annotated as [entity_value](entity_name), and assigns prefixes B, I, and 0 to each token
def get_tokens_with_entities(raw_text: str):
    # split the text by spaces (but not splitting the space inside the square brackets (so not splitting the "multi-word" entity value yet))
    raw_tokens = re.split(r"\s(?![^\[]*\])", raw_text)

    # a regex for matching the annotation according to our notation [entity_value](entity_name)
    entity_value_pattern = r"\[(?P<value>.+?)\]\((?P<entity>.+?)\)"

    # flags: re.IGNORECASE and re.MULTILINE
    entity_value_pattern_compiled = re.compile(entity_value_pattern, flags=re.I|re.M) # using it to compile a regular expression pattern provided as a string into a regex pattern object

    tokens_with_entities = []

    for raw_token in raw_tokens:
        match = entity_value_pattern_compiled.match(raw_token) # if no match then returns None

        if match:
            raw_entity_name, raw_entity_value = match.group("entity"), match.group("value")

            # we prefix the name of entity differently
            # B- indicates beginning of an entity
            # I- indicates the token is not a new entity itself but rather a part of existing one
            for i, raw_entity_token in enumerate(re.split("\s", raw_entity_value)):
                entity_prefix = "B" if i == 0 else "I"
                entity_name = f"{entity_prefix}-{raw_entity_name}"
                tokens_with_entities.append((raw_entity_token, entity_name))
        else:
            tokens_with_entities.append((raw_token, "O")) # no match

    return tokens_with_entities

# NLTK VERSION
def regex_from_term_nltk(term, lemmatizer): # TODO add some kind of automatic noun-verb-... identifier for lemmatization (so parts-of-speech required to add)
    regex = r"\b(" # Regex Opening
    tokensList = nltk.word_tokenize(term)

    # Adding terms to regex
    if len(tokensList) == 1: # in case of one-word term
        for token in tokensList:
            regex += token_cleaning(token, lemmatizer)

    else: # if it is a multi-word term
        decount = len(tokensList)
        for token in tokensList:
            decount = decount-1
            # add between-words
            if decount != len(tokensList)-1:
                regex+= r'\w*\W\w*\W*'
            # add token
            regex += token_cleaning(token, lemmatizer)

    regex += '''\w{0,5})(\W)''' # Regex Closure
    return regex

def token_cleaning(token, lemmatizer):
    token = token.lower()
    token = lemmatizer.lemmatize(token)
    return token

# Functions for document processing were taken from @https://github.com/shashankmc/eurovoc_entity_link/blob/master/EurovocTagger.py and were modified
def tsv_dic_processing(path):
    """
    :param path: the name of the eurovoc.tsv file
    :return: Dic: Dictionary in style of {ID: Word}
    :return: RevDic: Dictionary in style of {Word: ID}
    :return: list1: list of IDs
    :return: list2: list of words (concepts)
    """
    # Dic, RevDic, list1, list2
    # Only works with a 2-columns ([ID], [EN]) TSV file
    Dic = {}
    RevDic = {}
    list1 = []
    list2 = []
    with open(path, 'rt', encoding='utf8') as csvfile:
        myreader = csv.reader(csvfile, delimiter='\t')
        rcount = 0
        for row in myreader:
            rcount += 1
            ccount = 0
            if rcount > 1:
                for cells in row:
                    ccount += 1
                    if ccount ==1:
                        list1.append(cells)
                        key = cells
                    else:
                        list2.append(cells)
                        value = cells
                Dic[key] = value
                RevDic[value] = key
    return Dic, RevDic, list1, list2

def find_folder_with_type(given_path, doc_type): # returns all documents found in path
    doc_list = []
    for doc in os.listdir(given_path):
        if re.search (r'.*\%s$' % doc_type, doc) is not None: # even though this shows as error in IDE it's fine
            doc_list.append(doc)
    return doc_list


def folder_list_to_dic(given_path, given_list):
    dic = {}
    old_path = os.getcwd() # saving the previous working dir so we can switch back to that dir later
    os.chdir(given_path)

    # the input should be a list of file contained in a folder
    for file_name in given_list:
        print('importing', file_name, '...')
        with open("%s" % file_name, "r", encoding='utf8') as my_file:
            text = my_file.read()
        dic[file_name]= text

    os.chdir(old_path)
    return dic

# tagging by researching concept-regexed as a substring of the text (by using NLTK)
def tagging_document(path_of_tagged, given_doc_list, given_doc_dic, given_concept_list, given_eurovoc_reverse_dic):
    """
    This function takes the information of the descriptor (e.g., {id:concept}, id list, concept list, ...) and then with the given document information it creates the new tagged document in tagged folder. Additionally, it returns the new updated concept list which contains additional "concepts" found in the document text that seems to be related to one of the original concepts. Thus, expanding the vocabulary we have.

    :param path_of_tagged: the location (dir) of the tagged folder
    :param given_doc_list: a list of names of the documents
    :param given_doc_dic: a dic that contains the contents of the document i.e. {doc_name: doc_text}
    :param given_concept_list: the original concept list downloaded from Eurovoc
    :param given_eurovoc_reverse_dic: opposite of "given_concept_list" so {concept: id}
    :return: new_concept: this is the new expanded concept list
    """
    lemmatizer = WordNetLemmatizer()
    old_path = os.getcwd() # saving the previous working dir so we can switch back to that dir later
    os.chdir(path_of_tagged)
    new_concept_dic = given_eurovoc_reverse_dic.copy() # using the reverse eurovoc dict instead because can't add words with same id

    for doc_name in given_doc_list:
        tags_list=[]
        tagged_text = ""
        print('tagging', doc_name,'...')
        text = given_doc_dic[doc_name]
        text = text.lower()
        tagged_text = text # document's initial text

        # a concept tag will be done with a star (*), and the identifier with a +
        for concept in given_concept_list:

            if concept != "": # if concept empty, will tag everything (so need to make sure that it's not empty)
                # REGEX CREATION: creating regex of the concept such that it can be used to search in doc later
                regex = regex_from_term_nltk(concept, lemmatizer)

                # concept = concept.strip()
                # TAGGING #
                # semantically neutral symbols are chosen to prevent eurovoc concepts from matching tags
                if re.search(regex, text) is not None:
                    # these prints can be used to check performance
                    # print("Match made!")
                    # print("Found: " + re.search(regex, text).group() + ", for concept: " + concept)
                    match_in_text = re.search(regex, text).group()
                    if match_in_text not in given_concept_list:
                        # cleaning up the matched text
                        match_in_text = match_in_text.replace("\n", "")
                        match_in_text = match_in_text.strip()
                        match_in_text = match_in_text.strip(".,-")
                        new_concept_dic[match_in_text] = given_eurovoc_reverse_dic[concept]

                    tags_list.append(concept)
                    sub_regex = r"[" + concept + r"]"
                    sub_regex += r"(" + given_eurovoc_reverse_dic[concept] + r") " # insert the identifier
                    tagged_text = re.sub(regex, sub_regex, tagged_text)

    # create a new file with the tagged file
        file = open("%s_TAGGED.txt" % doc_name, "w", encoding='utf8')
        file.write(tagged_text)
        file.close()

    os.chdir(old_path) # change back to previous path

    return new_concept_dic

### Relations Extraction

In [69]:
# functions
def get_triples_stanford_openie(given_client, given_sentence):
    triples_list = []

    # returns dict in this style: {'subject': 'Obama', 'relation': 'was born in', 'object': 'Hawaii'}
    # for triple in tqdm(given_client.annotate(given_sentence)): # this can be used for debugging
    for triple in given_client.annotate(given_sentence):
        triples_list.append([triple.get("subject"), triple.get("relation"), triple.get("object")])

    return triples_list

def split_text_into_sentence(given_doc_names, given_docs):
    given_docs_tokenized = {}
    for doc_name in given_doc_names:
        text_of_doc = given_docs[doc_name]
        text_of_doc = text_of_doc.replace('\n',' ') # removing the newline string from text
        text_of_doc = text_of_doc.replace('\xad ','') # removing the hyphen used for line breaking

        text_of_doc_tokenized = sent_tokenize(text_of_doc)
        given_docs_tokenized[doc_name] = text_of_doc_tokenized

    return given_docs_tokenized

# this function is modified from "version1_simple" file
def create_kg_csv(subjects, predicates, objects, re_type, model_used):
    """
    [source(subject) --relation(predicate)--> target(object)]
    :param model_used:
    :param subjects: source
    :param predicates: relation
    :param objects: target
    :param re_type: currently, we have only relation extraction type of "simple" and "predefined_dictionary"
    :return: returns nothing only creates the csv file
    """
    # field names
    fields = ['Subject', 'Predicate', 'Object']
    filename = os.getcwd() + "\\triples_data\\" + re_type + "\\" + model_used + "\\kg_of_" + re_type + "_en.csv"

    rows = [[subjects[i], predicates[i], objects[i]] for i in range(len(subjects))]

    # find out empty and None strings, replacing it with "-"
    for i in range(len(rows)):
        for j in range(len(rows[0])): # so len 3
            if rows[i][j] == "" or rows[i][j] is None: rows[i][j] = "-"

    # writing to csv file
    with open(filename, 'w', newline = '') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(fields) # first writing fields
        csv_writer.writerows(rows) # now the remaining record

# German: de, French: fr, but can work for any other languages too
def simple_translator(language, re_type, model_used): # assumes that en kg csv is already created
    translator = Translator()

    filename = os.getcwd() + "\\triples_data\\" + re_type + "\\" + model_used
    filename_english = filename + "\\kg_of_" + re_type + "_en.csv"
    filename_assigned_language = filename + "\\kg_of_" + re_type + "_" + language + ".csv"

    with open(filename_english, mode ="r") as file_original, open(filename_assigned_language, mode = "w", newline = '') as file_2:
      csv_reader = csv.reader(file_original)
      csv_writer = csv.writer(file_2)

      for lines in csv_reader: # each line is a list of 3 elements (source - relation - target)
        source_en, relation_en, target_en = lines[0], lines[1], lines[2]

        # src(source) = english, dest(destination) = language to translate to
        translated_source, translated_relation, translated_target = translator.translate(source_en, src = "en", dest = language), translator.translate(relation_en, src = "en", dest = language), translator.translate(target_en, src = "en", dest = language)

        temp_row = [translated_source.text, translated_relation.text, translated_target.text]
        csv_writer.writerow(temp_row)

# German: de, French: fr, but can work for any other languages too
def simple_translator_with_concept(language, re_type, model_used, given_concept): # assumes that en kg csv is already created
    translator = Translator()

    filename = os.getcwd() + "\\triples_data\\" + re_type + "\\" + model_used
    filename_english = filename + "\\kg_of_" + re_type + "_en.csv"
    filename_assigned_language = filename + "\\kg_of_" + re_type + "_" + language + ".csv"

    with open(filename_english, mode ="r") as file_original, open(filename_assigned_language, mode = "w", newline = '') as file_2:
      csv_reader = csv.reader(file_original)
      csv_writer = csv.writer(file_2)

      for lines in csv_reader: # each line is a list of 3 elements (source - relation - target)
        source_en, relation_en, target_en = lines[0], lines[1], lines[2]

        # src(source) = english, dest(destination) = language to translate to
        translated_source, translated_relation, translated_target = translator.translate(source_en, src = "en", dest = language), translator.translate(relation_en, src = "en", dest = language), translator.translate(target_en, src = "en", dest = language)

        temp_row = [translated_source.text, translated_relation.text, translated_target.text]
        csv_writer.writerow(temp_row)

# Pipeline

In [6]:
# your own project path dir here
my_path = "C:\\Users\\dnaen\\PycharmProjects\\bachelor_thesis_23"

#### 1.Importing Dataset

In [126]:
# this has to be ran only once, because it creates the eurovoc_en.tsv file (which should already be there)
# create_tsv_of_language("en")

In [3]:
# to make sure that we are in the original working directory
data_path = my_path + "\\src\\main"
os.chdir(data_path)
print(os.getcwd()) # this should return something like "...\src\main"

NameError: name 'os' is not defined

In [75]:
tsv_file = "eurovoc_en.tsv"

# getting info of ids and concepts from the tsv file
eurovoc_dic, eurovoc_reverse_dic, id_list, concept_list = tsv_dic_processing(tsv_file)
print('Eurovoc imported')

Eurovoc imported


In [85]:
eurovoc_reverse_dic

{'AAMS countries': '594',
 'abandoned child': '759',
 'abandoned land': '4444',
 'ABM Agreement': '3509',
 'abolition of customs duties': '4333',
 'abortion': '4504',
 'Abruzzi': '5075',
 'absenteeism': '5339',
 'absolute majority': '1746',
 'abstentionism': '5984',
 'abuse of power': '186',
 'academic freedom': '3914',
 'access to a profession': '545',
 'access to Community information': '5399',
 'access to education': '280',
 'access to information': '453',
 'access to the courts': '5400',
 'accession criteria': '6706',
 'accession negotiations': '6708',
 'accession to an agreement': '5420',
 'accession to the European Union': '12',
 'accident in the home': '5314',
 'accident prevention': '5810',
 'accidental pollution': '6413',
 'account': '61',
 'accountant': '60',
 'accounting': '54',
 'accounting entry': '1333',
 'accounting system': '4362',
 'acculturation': '4873',
 'acid': '5035',
 'acid rain': '4165',
 'acidification': '6407',
 'acoustics': '3291',
 'ACP countries': '5083',
 

In [126]:
# Extracting all existing txt documents in the path
data_path = my_path + "\\data\\en\\directives_txt"
document_name_list = find_folder_with_type(data_path, '.txt') # detection of txt files in the folder
document_dic = folder_list_to_dic(data_path, document_name_list) # storing document content in a dictionary

importing Directive_(EU)_2016_1919_en.txt ...
importing Directive_(EU)_2016_343_en.txt ...
importing Directive_(EU)_2016_800_en.txt ...
importing Directive_2010_64_EU_en.txt ...
importing Directive_2012_13_EU_en.txt ...
importing Directive_2013_48_EU_en.txt ...


#### 2. Entity Extraction (NER)

In [177]:
# tagging document
os.chdir(my_path + "\\src\\main")
print(os.getcwd())
data_path = my_path + "\\data\\en\\directives_txt_tagged"
updated_concept_list = tagging_document(data_path, document_name_list, document_dic, concept_list, eurovoc_reverse_dic)
print(os.getcwd())


tagging Directive_(EU)_2016_1919_en.txt ...
tagging Directive_(EU)_2016_343_en.txt ...
tagging Directive_(EU)_2016_800_en.txt ...
tagging Directive_2010_64_EU_en.txt ...
tagging Directive_2012_13_EU_en.txt ...
tagging Directive_2013_48_EU_en.txt ...


In [179]:
# have to run this only once (if file "updated_eurovoc_en.tsv" exists no need to run it)
create_tsv_of_any_given_concept(updated_concept_list, "en")

#### 3. Relation Extraction (RE)

In [ ]:
# !pip install stanford_openie

In [4]:
# https://stanfordnlp.github.io/CoreNLP/openie.html#api
# Default value of openie.affinity_probability_cap was 1/3.
properties = {
    'openie.affinity_probability_cap': 2 / 3,
}

In [44]:
document_dic_tokenized = split_text_into_sentence(document_name_list, document_dic)
list_of_triples = []
with StanfordOpenIE(properties=properties) as client: # opening the server
    for doc_name in document_name_list:
        for sentence in document_dic_tokenized[doc_name]:
            current_triple_list_of_sentence = get_triples_stanford_openie(client, sentence) # returns [subject, relation, object]
            if current_triple_list_of_sentence: # only run the loop if list not empty
                for current_triple in current_triple_list_of_sentence: # a sentence can have multiple triples
                    list_of_triples.append(current_triple)
        print("Document " + doc_name + "finished")


Starting server with command: java -Xmx8G -cp C:\Users\dnaen\.stanfordnlp_resources\stanford-corenlp-4.5.3/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-7e1f36d3683141db.props -preload openie
Document Directive_2013_48_EU_en.txtfinished


In [49]:
# extract subject
source = [i[0] for i in list_of_triples]

# extract relation
relation = [i[1] for i in list_of_triples]

# extract object
target = [i[2] for i in list_of_triples]

#### 4. Build Knowledge Graph

In [66]:
data_path = my_path + "\\src\\main"
os.chdir(data_path)

create_kg_csv(source, relation, target, "predefined_dictionary", "stanford_OpenIE")

FALSE: C:\Users\dnaen\PycharmProjects\bachelor_thesis_23\src\main\triples_data\predefined_dictionary\stanford_OpenIE\kg_of_predefined_dictionary_en.csv
CORRET: C:\Users\dnaen\PycharmProjects\bachelor_thesis_23\src\main\triples_data\predefined_dictionary\stanford_OpenIE\kg_of_predefined-dictionary_en.csv


#### 5. Translate


In [73]:
simple_translator("de", "predefined_dictionary", "stanford_OpenIE") # CAUTION: since this uses API it can "timeout", so if happens just run it again

ConnectTimeout: timed out

In [184]:
os.getcwd()

'C:\\Users\\dnaen\\PycharmProjects\\bachelor_thesis_23\\data\\en\\directives_txt_tagged'

# Trash

In [0]:
# nltk.download('punkt') # unsupervised trainable model, which means it can be trained on unlabeled data (Data that has not been tagged with information identifying its characteristics, properties, or categories is referred to as unlabeled data.)

In [132]:
# the code below can be used to check what were the new concepts that were added
temp = list(updated_concept_list.values())
my_dict = {i: temp.count(i) for i in temp}
multiple_elements = []
for key, value in my_dict.items():
    if value > 1:
        multiple_elements.append(key)

for elem in multiple_elements:
    value = {i for i in updated_concept_list if updated_concept_list[i] == elem}
    print(str(value) + " in id: " + elem)
    print()